In [1]:
#  This is an adaptation of https://github.com/QML-HEP/qGAN/blob/main/1_classical_benchmark/ClassicalGAN.ipynb to work on our data set

In [2]:
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import numpy as np
import time
import csv

from math import *

from tools import *

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed);

Random Seed:  999


In [3]:
N=8 # number of futures 
nevt = 10000  # number of events used for training and testing
batch_size = 100

load_data = np.load(r"..\data preprocessing\4_features_minmax_npy_unsplit\x_minmax_sig.npy",allow_pickle=True)[:nevt]

In [4]:
X_train=load_data

dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train))
dataloader = torch.utils.data.DataLoader(dataset, batch_size= batch_size,shuffle=True)

real_data = np.mean(X_train, axis = 0) # Average of training set

In [5]:
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 0

#latent vec space dim
latent_dim = N
out_dim = N
hidden_dims = [256, 128, 64, 32, 16, 8]
#hidden_dims = [256, 128, 64, 32, 16] #N = 8

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [6]:
# flips a int array's values with some probability 
# Used for training
def bit_flip(x, prob=0.00):
    selection = np.random.uniform(0, 1, x.shape) < prob
    tmp = np.logical_not(x[selection])
    x[selection] = 1. * tmp.type(torch.FloatTensor) 
    return x

In [7]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.05)

In [8]:
# Generator class
class Generator(nn.Module):
    def __init__(self, ngpu, latent_dim):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        modules = []
        
        in_dim = latent_dim

        for l in hidden_dims : 
            modules.append(nn.Linear(in_dim, l))
            modules.append(nn.LeakyReLU(0.2))
            modules.append(nn.Dropout(0.2))

            in_dim = l

        modules.append(nn.Linear(in_dim, out_dim))
        modules.append(nn.LeakyReLU(0.2))

        self.main = nn.Sequential(*modules)

    def forward(self, input):
        return self.main(input)

In [9]:
# Create the generator
netG = Generator(ngpu, latent_dim).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Initialize weights randomly with mean=0, stdev=0.05.
netG.apply(weights_init)

# Print the model
print(netG)

Generator(
  (main): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=64, out_features=32, bias=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=32, out_features=16, bias=True)
    (13): LeakyReLU(negative_slope=0.2)
    (14): Dropout(p=0.2, inplace=False)
    (15): Linear(in_features=16, out_features=8, bias=True)
    (16): LeakyReLU(negative_slope=0.2)
    (17): Dropout(p=0.2, inplace=False)
    (18): Linear(in_features=8, out_features=8, bias=True)
    (19): LeakyReLU(negative_slope=0.2)
  )
)


In [10]:
# Discriminator class
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        modules = []        
        
        in_dim = out_dim

        for l in hidden_dims:
            modules.append(nn.Linear(in_dim,l))
            modules.append(nn.LeakyReLU(0.2))
            modules.append(nn.Dropout(0.2))
            in_dim = l
            
        modules.append(nn.Linear(in_dim,1))
        modules.append(nn.Sigmoid())

        self.main = nn.Sequential(*modules)

    def forward(self, input):
        return self.main(input)

In [11]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Initialize weights randomly with mean=0, stdev=0.05.
netD.apply(weights_init)

# Print the model
print(netD)

Discriminator(
  (main): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=64, out_features=32, bias=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=32, out_features=16, bias=True)
    (13): LeakyReLU(negative_slope=0.2)
    (14): Dropout(p=0.2, inplace=False)
    (15): Linear(in_features=16, out_features=8, bias=True)
    (16): LeakyReLU(negative_slope=0.2)
    (17): Dropout(p=0.2, inplace=False)
    (18): Linear(in_features=8, out_features=1, bias=True)
    (19): Sigmoid()
  )
)


In [12]:
# Loss function 
criterion = nn.BCELoss()

# Number of training epochs
num_epochs = 2000


# Create random noise vector used 
# to generate fake images at the end of each epoch
fixed_noise = torch.randn(len(X_train), latent_dim, device=device)

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

# Learning rate for optimizers
lr = 0.00002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Flip probability for real and fake labels
flip_rate = 0.00

# Path to save 
save_folder = "result"

# Crate the directory to save results
if save_folder is not None : 
    create_folder(save_folder)   

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [13]:
# Training
img_list = []
G_losses = []
D_losses = []
start = time.time()

results = []

MSE = []

if save_folder is not None : 
    with open(os.path.join(save_folder, 'output.csv'), mode='w') as csv_file:
        fieldnames = ['epoch', 'd_loss', 'g_loss', 'mean_image' 'MSE']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        
print("Starting Training Loop...")

# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network for real data: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch

        netD.zero_grad()

        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        
        label = torch.full((b_size,), real_label, device=device).type(torch.FloatTensor)
        
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, bit_flip(label, prob = flip_rate))
        # Calculate gradients for D in backward pass
        errD_real.backward()
        
        D_x = output.mean().item()
        
        # Update for real data
        optimizerD.step()
        
        ## Train with all-fake batch
        # Generate batch of latent vectors
        netD.zero_grad()
        
        noise = torch.randn(b_size, latent_dim, device=device)
        
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, bit_flip(label, prob = flip_rate))
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        # Update D
        optimizerD.step()
        
        errD = (errD_real.item() + errD_fake.item())/2.

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        
        
        label.fill_(real_label)  # fake labels are real for generator cost
        
        
        netG.zero_grad()
        
        noise = torch.randn(b_size, latent_dim, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        
        
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG1 = criterion(output, bit_flip(label, prob = flip_rate))
        # Calculate gradients for G
        errG1.backward()
        
        # Update G
        optimizerG.step()
        
        
        netG.zero_grad()
        
        noise = torch.randn(b_size, latent_dim, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG2 = criterion(output, bit_flip(label, prob = flip_rate))
        # Calculate gradients for G
        errG2.backward()
        
        # Update G
        optimizerG.step()
        
        errG = (errG1.item() + errG2.item())/2.0
        
        D_G_z2 = output.mean().item()
        

        # Output training stats
        a = np.mean(fake.detach().numpy(), axis = 0)
        MSE.append(np.sum((a - real_data)**2))

    # Pring results                    
    if epoch % 10 == 0:
        print('[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f/%.4f\t MSE :%.4f\t time taken : %.4f min'
                % (epoch, num_epochs,
                    errD, errG, D_x, D_G_z1, D_G_z2, MSE[-1], (time.time() - start)/60.0))
        start  = time.time() 

        # Save Losses for plotting later
        G_losses.append(errG)
        D_losses.append(errD)
    with torch.no_grad():
            fake = netG(fixed_noise).detach().cpu().numpy()

    # Check how the generator is doing by saving G's output on fixed_noise
    if epoch % 100 == 0:        
        # Plot real and generated images
        # num_to_plot = Number of output images to plot
        plot_outputs(save_folder, np.mean(X_train, axis = 0), fake, epoch, num_to_plot = 100)
        

                
    if save_folder is not None :
        with open(os.path.join(save_folder, 'output.csv'), mode='a') as csv_file:
            fieldnames = ['epoch', 'd_loss', 'g_loss', 'mean_image', 'MSE']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writerow({'epoch': epoch, 'd_loss': np.around(D_losses[-1], 5),
                                'g_loss': np.around(G_losses[-1], 5),
                                'mean_image' : np.mean(fake, axis = 0),
                                'MSE': np.around(MSE[-1], 5)})
        torch.save({
                'epoch': epoch,
                'model_state_dict': netG.state_dict(),
                'optimizer_state_dict': optimizerG.state_dict(),
                'loss' : G_losses[-1]
                }, os.path.join(save_folder, "Generator_model.pt"))

        torch.save({
                'epoch': epoch,
                'model_state_dict': netD.state_dict(),
                'optimizer_state_dict': optimizerD.state_dict(),
                'loss' : D_losses[-1]
                }, os.path.join(save_folder, "Discriminator_model.pt"))

if save_folder is not None : 
    plot_loss(save_folder, G_losses, D_losses)
    plot_error(save_folder, MSE)

Starting Training Loop...
[0/2000]	Loss_D: 0.7068	Loss_G: 0.8731	D(x): 0.4177	D(G(z)): 0.4176/0.4177	 MSE :0.0887	 time taken : 0.0420 min


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 720x864 with 2 Axes>